In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import seaborn as sns; sns.set(style="ticks", color_codes="pastel")
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import tree
import time
from sklearn import metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss


def stdscaler(df):
    scaler = StandardScaler()
    scaler.fit(df)
    scX_df = scaler.transform(df)
    df=pd.DataFrame(scX_df, index =df.index,columns =df.columns  )
    return df


import time

def kfoldtest(kfoldn,model,Xtrain_df,ytrain_df):
    kf=StratifiedKFold(n_splits=kfoldn)
    accu=[]
    auc=[]
    f1 =[]
    aab =[]
    dcd = []
    tim = []
    kf.get_n_splits(Xtrain_df,ytrain_df)
    
    for train_index, test_index in kf.split(Xtrain_df,ytrain_df):
        #print("TRAIN:", len(train_index), "TEST:", len(test_index))
        t1 =time.time()
        X_train, X_test = Xtrain_df.iloc[train_index], Xtrain_df.iloc[test_index]
        y_train, y_test = ytrain_df.iloc[train_index], ytrain_df.iloc[test_index]
        model.fit(X_train,y_train)
        try:
            accu.append(model.score(X_test,y_test))
        except:
            print("This model have no build-in score.")
            
        auc.append(roc_auc_score(y_test,model.predict(X_test)))
        f1.append(f1_score(y_test, model.predict(X_test).astype(int),average='binary'))
        tim.append(time.time()-t1)
        cm1=confusion_matrix(test_y, clf.predict(test_data),labels = [1,0])
        sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
        aab.append(sensitivity1)

        specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
        dcd.append(specificity1)
    return accu,auc,f1,aab,dcd,tim

sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})
file_name = 'model1.csv'
data =pd.read_csv(file_name,index_col = 0)
print(data.columns)


Index(['paystate', 'lastpay', 'billpay', 'billlim', 'paylim', 'billchangelim0',
       'paychangelim0', 'billchangelim1', 'paychangelim1', 'billchangelim2',
       'paychangelim2', 'billchangelim3', 'paychangelim3', 'billchangelim4',
       'paychangelim4', 'ageclass_20s', 'ageclass_30s', 'ageclass_40s',
       'ageclass_50s', 'ageclass_60s', 'ageclass_70s', 'sex_1', 'sex_2',
       'education_0', 'education_1', 'education_2', 'education_3',
       'education_4', 'education_5', 'education_6', 'marriage_0', 'marriage_1',
       'marriage_2', 'marriage_3', 'default.payment.next.month'],
      dtype='object')


In [6]:
v1 = data[data["default.payment.next.month"]==1].copy()
v0 = data[data["default.payment.next.month"]==0].copy()
v1.rename(columns={"default.payment.next.month": "default"},inplace = True)
v0.rename(columns={"default.payment.next.month": "default"},inplace = True)
y1 = v1["default"].copy()
x1 = v1.copy()
del x1["default"]
y0 = v0["default"].copy()
x0 = v0.copy()
del x0["default"]

tr1,te1,try1,tey1 = train_test_split(
    x1, y1, test_size=0.20, random_state=42)
tr2,te2,try2,tey2 = train_test_split(
    x0, y0, test_size=0.20, random_state=42)
####
#tr2=tr2.sample(len(tr1.index))

#try2 = try2.loc[tr2.index].copy()
####


train_data=pd.concat([tr1,tr2],axis=0)

train_y = pd.concat([try1,try2],axis=0)
test_data=pd.concat([te1,te2],axis=0)

test_y = pd.concat([tey1,tey2],axis=0)


train_data = stdscaler(train_data )
test_data = stdscaler(test_data )


In [7]:

"""pca =PCA(n_components=.99).fit(train_data)
train_data = pd.DataFrame(pca.transform(train_data),index = train_data.index)   

test_data= pd.DataFrame(pca.transform(test_data),index = test_data.index)  
"""

'pca =PCA(n_components=.99).fit(train_data)\ntrain_data = pd.DataFrame(pca.transform(train_data),index = train_data.index)   \n\ntest_data= pd.DataFrame(pca.transform(test_data),index = test_data.index)  \n'

In [8]:

from sklearn.model_selection import GridSearchCV
param_grid  = { 'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
                'activation': ['tanh', 'relu'],
                'solver': ['sgd', 'adam'],
                'alpha': [0.0001, 0.05],
                'learning_rate': ['constant','adaptive'],
               
                }

clf = GridSearchCV(MLPClassifier(max_iter=3000), param_grid, cv = 5, scoring = 'roc_auc',n_jobs=-1)


clf.fit(train_data,train_y)
#let's see the best estimator
print(clf.best_estimator_)
#with its score
print('CV accuarcy:',np.abs(clf.best_score_))
print(classification_report(test_y, clf.predict(test_data), labels=[1,0]))
print(''.join([c for c in str(clf) if c.isupper()])[:3]+' area='+str(roc_auc_score(test_y, clf.predict(test_data)))[:4])



MLPClassifier(activation='tanh', alpha=0.05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 100, 50), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=3000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
CV accuarcy: 0.7603952171904297
              precision    recall  f1-score   support

           1       0.62      0.38      0.47      1264
           0       0.84      0.93      0.89      4563

    accuracy                           0.81      5827
   macro avg       0.73      0.66      0.68      5827
weighted avg       0.80      0.81      0.80      5827

GSC area=0.65


In [10]:
param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [5,10,20]
             }


DTC = DecisionTreeClassifier(random_state = 0, 
                             max_features = "auto", class_weight = "balanced",
                             max_depth = None)

ABC = AdaBoostClassifier(base_estimator = DTC)

# run grid search
grid_search_ABC = GridSearchCV(ABC, param_grid=param_grid,cv = 5, scoring = 'roc_auc',n_jobs=-1)

grid_search_ABC.fit(train_data,train_y)
#let's see the best estimator
print(grid_search_ABC.best_estimator_)
#with its score
print('CV accuarcy:',np.abs(grid_search_ABC.best_score_))
print(classification_report(test_y, grid_search_ABC.predict(test_data), labels=[1,0]))


AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight='balanced',
                                                         criterion='entropy',
                                                         max_depth=None,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
            

In [ ]:
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.05, 0.15,0.2 ],
    "min_samples_split": [0.1,0,25,0.5],
    "min_samples_leaf": [0.1,0.25,0.5],
    "max_depth":[3,5,8,10],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 1.0],
    "n_estimators":[10]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=5, scoring = 'roc_auc',n_jobs=-1)

clf.fit(train_data,train_y)
print(clf.best_estimator_)
print(clf.best_params_)
print('CV accuarcy:',np.abs(clf.best_score_))
print(classification_report(test_y, clf.predict(test_data), labels=[1,0]))
print(''.join([c for c in str(clf) if c.isupper()])[:3]+' area='+str(roc_auc_score(test_y, clf.predict(test_data)))[:4])


In [12]:
param_grid = { 
    'n_estimators': [10,50,100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [5,8,10,50],
    'criterion' :['gini', 'entropy']
    }


clf = GridSearchCV(RandomForestClassifier(random_state=42),param_grid, cv= 5,scoring = 'roc_auc',n_jobs=-1)
clf.fit(train_data,train_y)
print(clf.best_estimator_)
#with its score
print('CV accuarcy:',np.abs(clf.best_score_))
print(classification_report(test_y, clf.predict(test_data), labels=[1,0]))
print(''.join([c for c in str(clf) if c.isupper()])[:3]+' area='+str(roc_auc_score(test_y, clf.predict(test_data)))[:4])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
CV accuarcy: 0.7743908334671861
              precision    recall  f1-score   support

           1       0.67      0.21      0.32      1264
           0       0.82      0.97      0.89      4563

    accuracy                           0.81      5827
   macro avg       0.74      0.59      0.60      5827
weighted avg       0.78      0.81      0.76      5827

GSC area=0.58
